In [ ]:
import math

import ROOT

from uits3_krakow22.src.Event import Event
from uits3_krakow22.src.Cluster import Cluster
from uits3_krakow22.src.Track import Track
from uits3_krakow22.src.Vertex import Vertex
from uits3_krakow22.src.Utils import *

from ipywidgets import IntProgress
from IPython.display import display,HTML,clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

### Select events

In [ ]:
path = "/home/berki/Software/uits3_krakow22/run456195948_221112200957_clusters_2.pkl"
nEvents = -1
selectedEvents = []
if nEvents > 0 :
    f = IntProgress(min=0, max=nEvents, description="Processing:")
    display(f)

for i,event in enumerate(readEvents(path, nEvents=nEvents)):
    if nEvents > 0 and i>0 and i%1e4 == 0: f.value += 1e4
    eventStatus = True
    for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]:
        clusters = event.selectDetector(alpide)
        selectedClusters = [cluster for cluster in clusters if 15 > cluster.size > 7]
        if len(selectedClusters) != 1:
            eventStatus = False
            break
    if eventStatus : selectedEvents.append(event)
        
if f : f.bar_style = "success"
print("Found events:",len(selectedEvents),", Fraction:",len(selectedEvents)*100/nEvents,"%")

#### Save selected events to file

In [ ]:
path_selected = "/home/berki/Software/uits3_krakow22/selectedEvents.pkl"
save2pickle(selectedEvents,path_selected)

### Add tracks and vertex to events

In [ ]:
events= []
for event in readEvents(path_selected):
    TrackLeft = Track()
    TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
    TrackRight = Track()
    TrackRight.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
    vertex = Vertex()
    vertex.fromTracks([TrackLeft,TrackRight])
    event.addTrack(TrackLeft)
    event.addTrack(TrackRight)
    event.vertex = vertex
    events.append(event)

#### Save tracked events to file

In [ ]:
path_tracked = "/home/berki/Software/uits3_krakow22/trackedEvents.pkl"
save2pickle(events,path_tracked)

#### Opening angle

In [ ]:
%jsroot on
hOpeningAngle = ROOT.TH1F("Angle","Angle",300,60,120)
for event in events:
    hOpeningAngle.Fill(event.vertex.openingAngle*360/(2*math.pi))
canvas = ROOT.TCanvas()
hOpeningAngle.Draw("COLZ")
canvas.Draw()

#### DCA between two tracks

In [ ]:
%jsroot on
hDCA = ROOT.TH1F("DCA","DCA",600,-1,30)
for event in events:
    if abs(event.vertex.openingAngle - math.pi/2) < 0.05:
        hDCA.Fill(event.vertex.dca)
canvas = ROOT.TCanvas()
hDCA.Draw()
canvas.Draw()

#### DCA between vertex and [0,0,0]
Should probably use [0,0,-10.2] as origin as 0 along beam axis is currently the middle of the first layer

In [ ]:
%jsroot on
hDCA2origin = ROOT.TH1F("DCAtoOrigin","DCA to [0,0,0]",400,0,40)
for event in events:
    #if abs(event.vertex.openingAngle - math.pi/2) < 0.05:
    hDCA2origin.Fill(event.vertex.dca2origin)
canvas = ROOT.TCanvas()
hDCA2origin.Draw()
canvas.Draw()

#### Vertex positions

In [ ]:
%jsroot on
hVertex = ROOT.TH3F("VertexXYZ","VertexXYZ",100,-6,6, 100,-6,6, 100,-24,-12)
for event in events:
    if abs(event.vertex.openingAngle - math.pi/2) < 0.05 and event.vertex.dca < 2:
        hVertex.Fill(event.vertex.point[0],event.vertex.point[1],event.vertex.point[2])
    
canvas = ROOT.TCanvas()
hVertex.Draw()
canvas.Draw()

In [ ]:
%jsroot on
hVertexXY = ROOT.TH2F("VertexXY","VertexXY",200,-20,20,200,-40,0)
hVertexXZ = ROOT.TH2F("VertexXZ","VertexXZ",200,-20,20,200,-20,20)
hVertexYZ = ROOT.TH2F("VertexYZ","VertexYZ",200,-40,0,200,-20,20)
for event in events:
    if abs(event.vertex.openingAngle - math.pi/2) < 0.05 and event.vertex.dca < 2:
        hVertexXY.Fill(-event.vertex.point[0],event.vertex.point[2])
        hVertexXZ.Fill(-event.vertex.point[0],event.vertex.point[1])
        hVertexYZ.Fill(event.vertex.point[2],event.vertex.point[1])
    
canvas = ROOT.TCanvas("","",1200,400)
canvas.Divide(3)
canvas.cd(1)
hVertexXY.Draw("COLZ")
canvas.cd(2)
hVertexXZ.Draw("COLZ")
canvas.cd(3)
hVertexYZ.Draw("COLZ")
canvas.Draw()

#### Position shift between layers

In [ ]:
%jsroot on
alpide1 = "ALPIDE_0"
alpide2 = "ALPIDE_1"
hShiftXY = ROOT.TH2F("ShiftXY","ShiftXY",150,-15,15,75,-7.5,7.5)
hShiftX = ROOT.TH1F("ShiftX","ShiftX",150,-15,15)
hShiftY = ROOT.TH1F("ShiftY","ShiftY",75,-7.5,7.5)
for event in events:
    if abs(event.vertex.openingAngle - math.pi/2) < 0.05 and event.vertex.dca < 2:
        hShiftXY.Fill(event.selectDetector(alpide1)[0].localPos[0]-event.selectDetector(alpide2)[0].localPos[0],
                      event.selectDetector(alpide1)[0].localPos[1]-event.selectDetector(alpide2)[0].localPos[1])
        hShiftX.Fill(event.selectDetector(alpide1)[0].localPos[0]-event.selectDetector(alpide2)[0].localPos[0])
        hShiftY.Fill(event.selectDetector(alpide1)[0].localPos[1]-event.selectDetector(alpide2)[0].localPos[1])
canvas = ROOT.TCanvas("","",1200,400)
canvas.Divide(3)
canvas.cd(1)
hShiftXY.Draw("COLZ")
canvas.cd(2)
hShiftX.Draw()
canvas.cd(3)
hShiftY.Draw()
canvas.Draw()

#### Correlation between ALPIDEs after all selections

In [ ]:
%jsroot on

nBins = {"x-Axis" : 600, "y-Axis" : 300}
nRange = {"x-Axis" : 15, "y-Axis" : 7.5}
axis = {"x-Axis":0, "y-Axis":1}

detector1 = "ALPIDE_0"
detector2 = "ALPIDE_4"
            
hCorrXX = ROOT.TH2F("Corr_"+detector1+"_x-Axis_"+detector2+"_x-Axis","Corr_"+detector1+"_x-Axis_"+detector2+"_x-Axis", nBins.get("x-Axis"),-nRange.get("x-Axis"),nRange.get("x-Axis"),nBins.get("x-Axis"),-nRange.get("x-Axis"),nRange.get("x-Axis"))
hCorrYY = ROOT.TH2F("Corr_"+detector1+"_y-Axis_"+detector2+"_y-Axis","Corr_"+detector1+"_y-Axis_"+detector2+"_y-Axis", nBins.get("y-Axis"),-nRange.get("y-Axis"),nRange.get("y-Axis"),nBins.get("y-Axis"),-nRange.get("y-Axis"),nRange.get("y-Axis"))
hCorrXX.SetXTitle(detector1 + " - x-Axis")
hCorrXX.SetYTitle(detector2 + " - x-Axis")
hCorrYY.SetXTitle(detector1 + " - y-Axis")
hCorrYY.SetYTitle(detector2 + " - y-Axis")

for event in events:
    if abs(event.vertex.openingAngle - math.pi/2) < 0.05 and event.vertex.dca < 2:
        for cluster1 in event.selectDetector(detector1):
            for cluster2 in event.selectDetector(detector2):
                hCorrXX.Fill(cluster1.localPos[0],cluster2.localPos[0])
                hCorrYY.Fill(cluster1.localPos[1],cluster2.localPos[1])
                

canvas = ROOT.TCanvas("","",1000,500)
canvas.Divide(2)
canvas.cd(1)
hCorrXX.Draw("COLZ")
canvas.cd(2)
hCorrYY.Draw("COLZ")
canvas.Draw()